In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import random
import glob
import csv
import pandas as pd
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.metrics import structural_similarity as ssim

In [ ]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        self.double_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)

class Encoder(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.enc1 = DoubleConv(in_channels, 64)
        self.enc2 = DoubleConv(64, 128)
        self.enc3 = DoubleConv(128, 256)
        self.enc4 = DoubleConv(256, 512)
        self.pool = nn.MaxPool2d(2, 2)

    def forward(self, x):
        enc1 = self.enc1(x)
        enc2 = self.enc2(self.pool(enc1))
        enc3 = self.enc3(self.pool(enc2))
        enc4 = self.enc4(self.pool(enc3))
        return enc1, enc2, enc3, enc4

class Bottleneck(nn.Module):
    def __init__(self):
        super().__init__()
        self.bottleneck = nn.Sequential(
            nn.MaxPool2d(2, 2),
            nn.Conv2d(512, 1024, kernel_size=3, padding=1),
            nn.BatchNorm2d(1024),
            nn.ReLU(inplace=True),
            nn.Conv2d(1024, 1024, kernel_size=3, padding=1),
            nn.BatchNorm2d(1024),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.bottleneck(x)

class Decoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.upconv4 = nn.ConvTranspose2d(1024, 512, 2, stride=2)
        self.dec4 = DoubleConv(1024, 512)
        self.upconv3 = nn.ConvTranspose2d(512, 256, 2, stride=2)
        self.dec3 = DoubleConv(512, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, 2, stride=2)
        self.dec2 = DoubleConv(256, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.dec1 = DoubleConv(128, 64)

    def forward(self, bottleneck, skip1, skip2, skip3, skip4):
        dec4 = self.upconv4(bottleneck)
        dec4 = torch.cat([dec4, skip4], dim=1)
        dec4 = self.dec4(dec4)

        dec3 = self.upconv3(dec4)
        dec3 = torch.cat([dec3, skip3], dim=1)
        dec3 = self.dec3(dec3)

        dec2 = self.upconv2(dec3)
        dec2 = torch.cat([dec2, skip2], dim=1)
        dec2 = self.dec2(dec2)

        dec1 = self.upconv1(dec2)
        dec1 = torch.cat([dec1, skip1], dim=1)
        dec1 = self.dec1(dec1)
        return dec1

class UNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):  # Ubah ke 1 channel
        super().__init__()
        self.encoder = Encoder(in_channels)
        self.bottleneck = Bottleneck()
        self.decoder = Decoder()
        self.final_conv = nn.Conv2d(64, out_channels, 1)

    def forward(self, x):
        skip1, skip2, skip3, skip4 = self.encoder(x)
        bottleneck = self.bottleneck(skip4)
        decoded = self.decoder(bottleneck, skip1, skip2, skip3, skip4)
        return self.final_conv(decoded)

In [ ]:
# Custom Dataset
class SatelliteDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.images = glob.glob(os.path.join(image_dir, "*.jpg")) + \
                     glob.glob(os.path.join(image_dir, "*.png"))

    def add_noise(self, image):
        # Menambahkan berbagai jenis noise
        # noise_type = random.choice(['gaussian', 'salt_pepper', 'speckle'])
        noise_type = 'salt_pepper'

        if noise_type == 'gaussian':
            # Gaussian noise
            mean = 0
            sigma = random.uniform(0.02, 0.1)
            noise = torch.normal(mean, sigma, image.shape)
            noisy_image = image + noise

        elif noise_type == 'salt_pepper':
            # Salt and pepper noise
            prob = random.uniform(0.02, 0.1)
            noise = torch.rand(image.shape)
            salt = (noise > 1 - prob/2).float()
            pepper = (noise < prob/2).float()
            noisy_image = image * (1 - salt - pepper) + salt

        else:  # Speckle
            # Speckle noise
            intensity = random.uniform(0.05, 0.2)
            noise = torch.randn(image.shape) * intensity
            noisy_image = image + image * noise

        # Clip values to valid range [0, 1]
        return torch.clamp(noisy_image, 0, 1)

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = self.images[idx]
        # Baca gambar sebagai grayscale langsung
        image = Image.open(img_path).convert('L')

        if self.transform:
            image = self.transform(image)

        # Original image sebagai target (clean)
        target = image.clone()
        # Menambahkan noise pada input image
        noisy_image = self.add_noise(image)

        # Menambah dimensi channel (1 channel untuk grayscale)
        noisy_image = noisy_image.unsqueeze(0) if noisy_image.dim() == 2 else noisy_image
        target = target.unsqueeze(0) if target.dim() == 2 else target

        return noisy_image, target

In [ ]:
# Function for testing
def test_model(model, test_loader, device, output_csv):
    model.eval()
    psnr_list = []
    ssim_list = []
    results = []

    with torch.no_grad():
        for idx, (noisy_imgs, target_imgs) in enumerate(tqdm(test_loader, desc="Testing")):
            noisy_imgs = noisy_imgs.to(device)
            target_imgs = target_imgs.to(device)

            # Model prediction
            outputs = model(noisy_imgs)

            for i in range(noisy_imgs.size(0)):  # Per-image testing
                # Convert tensors to numpy
                pred = outputs[i].cpu().squeeze().numpy()
                target = target_imgs[i].cpu().squeeze().numpy()

                # Compute PSNR and SSIM
                psnr_value = psnr(target, pred, data_range=1.0)
                ssim_value, _ = ssim(target, pred, data_range=1.0, full=True)

                # Print PSNR and SSIM for each image
                print(f"Image {idx * test_loader.batch_size + i}: PSNR = {psnr_value:.4f}, SSIM = {ssim_value:.4f}")

                # Save results
                psnr_list.append(psnr_value)
                ssim_list.append(ssim_value)
                results.append({"Image Index": idx * test_loader.batch_size + i,
                                "PSNR": psnr_value,
                                "SSIM": ssim_value})

    # Save to CSV
    with open(output_csv, 'w', newline='') as csvfile:
        fieldnames = ['Image Index', 'PSNR', 'SSIM']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(results)

    # Calculate averages
    avg_psnr = sum(psnr_list) / len(psnr_list)
    avg_ssim = sum(ssim_list) / len(ssim_list)

    print(f"\nAverage PSNR: {avg_psnr:.4f}")
    print(f"Average SSIM: {avg_ssim:.4f}")

    return avg_psnr, avg_ssim

# Main testing pipeline
def main():
    # Set device
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f'Using device: {device}')

    # Hyperparameters
    BATCH_SIZE = 8
    IMAGE_SIZE = 256

    # Transform
    transform = transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor(),
    ])

    # Load test dataset
    test_dataset = SatelliteDataset(
        image_dir='/content/drive/MyDrive/Dataset/satellit_dataset/Test',
        transform=transform
    )

    test_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=2
    )

    # Load model
    model = UNet(in_channels=1, out_channels=1).to(device)
    checkpoint = torch.load('/content/drive/MyDrive/Dataset/satellit_dataset/model_saves/best_model.pth', map_location=device)
    model.load_state_dict(checkpoint['model_state_dict'])

    # Path to save results
    output_csv = '/content/drive/MyDrive/Dataset/satellit_dataset/test_results.csv'

    # Test model
    avg_psnr, avg_ssim = test_model(model, test_loader, device, output_csv)

    print("Testing completed!")
    print(f"Results saved to: {output_csv}")

if __name__ == "__main__":
    main()

Using device: cuda


<ipython-input-5-45d8a2923b92>:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('/content/drive/MyDrive/Dataset/satellit_dataset/model_saves/best_mo

Image 0: PSNR = 44.4984, SSIM = 0.9849
Image 1: PSNR = 42.7174, SSIM = 0.9926
Image 2: PSNR = 38.9391, SSIM = 0.9884
Image 3: PSNR = 38.0233, SSIM = 0.9719
Image 4: PSNR = 39.5832, SSIM = 0.9851
Image 5: PSNR = 39.1911, SSIM = 0.9840
Image 6: PSNR = 39.0860, SSIM = 0.9826
Image 7: PSNR = 42.7302, SSIM = 0.9854
Image 8: PSNR = 37.0237, SSIM = 0.9846
Image 9: PSNR = 33.2261, SSIM = 0.9758
Image 10: PSNR = 39.0801, SSIM = 0.9877
Image 11: PSNR = 40.1755, SSIM = 0.9784
Image 12: PSNR = 41.0445, SSIM = 0.9920
Image 13: PSNR = 43.8359, SSIM = 0.9925
Image 14: PSNR = 37.0724, SSIM = 0.9885
Image 15: PSNR = 37.1225, SSIM = 0.9865


Testing:   2%|▏         | 5/250 [00:42<30:21,  7.44s/it]  

Image 16: PSNR = 38.5669, SSIM = 0.9865
Image 17: PSNR = 43.6690, SSIM = 0.9827
Image 18: PSNR = 38.6881, SSIM = 0.9812
Image 19: PSNR = 36.2515, SSIM = 0.9890
Image 20: PSNR = 37.8001, SSIM = 0.9862
Image 21: PSNR = 39.7781, SSIM = 0.9814
Image 22: PSNR = 34.1390, SSIM = 0.9820
Image 23: PSNR = 37.2928, SSIM = 0.9793
Image 24: PSNR = 41.1251, SSIM = 0.9925
Image 25: PSNR = 32.2810, SSIM = 0.9669
Image 26: PSNR = 37.0362, SSIM = 0.9852
Image 27: PSNR = 40.1142, SSIM = 0.9598
Image 28: PSNR = 37.2646, SSIM = 0.9850
Image 29: PSNR = 43.7586, SSIM = 0.9849
Image 30: PSNR = 41.2517, SSIM = 0.9859
Image 31: PSNR = 37.9125, SSIM = 0.9876
Image 32: PSNR = 34.6585, SSIM = 0.8956
Image 33: PSNR = 38.8778, SSIM = 0.9835
Image 34: PSNR = 37.6136, SSIM = 0.9563
Image 35: PSNR = 37.1536, SSIM = 0.9897
Image 36: PSNR = 37.8359, SSIM = 0.9880
Image 37: PSNR = 36.6177, SSIM = 0.9828
Image 38: PSNR = 44.6611, SSIM = 0.9925
Image 39: PSNR = 39.1420, SSIM = 0.9890


...

Image 1962: PSNR = 44.1204, SSIM = 0.9752
Image 1963: PSNR = 35.6277, SSIM = 0.9803
Image 1964: PSNR = 45.7447, SSIM = 0.9894
Image 1965: PSNR = 36.2229, SSIM = 0.9716
Image 1966: PSNR = 42.5295, SSIM = 0.9866
Image 1967: PSNR = 38.0576, SSIM = 0.9713
Image 1968: PSNR = 42.3118, SSIM = 0.9888
Image 1969: PSNR = 49.3085, SSIM = 0.9919
Image 1970: PSNR = 41.9639, SSIM = 0.9920
Image 1971: PSNR = 45.0357, SSIM = 0.9734
Image 1972: PSNR = 40.3966, SSIM = 0.9899
Image 1973: PSNR = 40.5168, SSIM = 0.9877
Image 1974: PSNR = 37.7460, SSIM = 0.9776
Image 1975: PSNR = 45.0556, SSIM = 0.9869


Testing: 100%|██████████| 250/250 [01:18<00:00,  4.82it/s]

Image 1976: PSNR = 38.9338, SSIM = 0.9880
Image 1977: PSNR = 35.7103, SSIM = 0.9776
Image 1978: PSNR = 40.6869, SSIM = 0.9381
Image 1979: PSNR = 36.4940, SSIM = 0.9829
Image 1980: PSNR = 41.5735, SSIM = 0.9784
Image 1981: PSNR = 42.9190, SSIM = 0.9902
Image 1982: PSNR = 32.8766, SSIM = 0.9736
Image 1983: PSNR = 43.0481, SSIM = 0.9784
Image 1984: PSNR = 36.6000, SSIM = 0.9905
Image 1985: PSNR = 38.2223, SSIM = 0.9899
Image 1986: PSNR = 38.9987, SSIM = 0.9884
Image 1987: PSNR = 41.5119, SSIM = 0.9912
Image 1988: PSNR = 35.6721, SSIM = 0.9882
Image 1989: PSNR = 43.4636, SSIM = 0.9928
Image 1990: PSNR = 40.4391, SSIM = 0.7971
Image 1991: PSNR = 38.4643, SSIM = 0.9464
Image 1992: PSNR = 41.3994, SSIM = 0.9873
Image 1993: PSNR = 42.3622, SSIM = 0.9901
Image 1994: PSNR = 37.9432, SSIM = 0.9821
Image 1995: PSNR = 37.6130, SSIM = 0.9797
Image 1996: PSNR = 36.5046, SSIM = 0.9817
Image 1997: PSNR = 45.6439, SSIM = 0.9839
Image 1998: PSNR = 43.9445, SSIM = 0.9942
Image 1999: PSNR = 45.2957, SSIM =

Testing: 100%|██████████| 250/250 [01:18<00:00,  3.17it/s]


Average PSNR: 39.7892
Average SSIM: 0.9814
Testing completed!
Results saved to: /content/drive/MyDrive/Dataset/satellit_dataset/test_results.csv
